In [ ]:
import time

user = {}
daily_reward_claimed = False  # NEW FEATURE: daily reward check

responses = {
    "hi": "Hey bestie! 👋",
    "hello": "Hey bestie! 👋",
    "hey": "Hey bestie! 👋",
    "how are you": "Just vibing, you? 😎",
    "bye": "Aww, see you soon 💖",
}

state = {
    "energy": 80,
    "mood": "neutral",
    "points": 0,
    "room_level": 1,
    "decor_items": [],
}

room_descriptions = {
    1: "Simple room — just vibes.",
    2: "Getting cozier — nice progress!",
    3: "Your room is leveling up!🔥",
    4: "Aesthetic queen/king in the making!",
    5: "GenZ TikTok-worthy masterpiece ✨",
    6: "Your room looks like Pinterest 💗",
    7: "Ultra aesthetic influencer room 🌟",
}

shop_items = {
    "poster": {"cost": 10, "desc": "Cute artistic poster — +mood"},
    "plants": {"cost": 15, "desc": "Aesthetic plants — chill vibes"},
    "lamp": {"cost": 20, "desc": "Cozy lamp — better energy"},
    "beanbag": {"cost": 25, "desc": "Comfy beanbag — relax mode"},
    "rgb lights": {"cost": 35, "desc": "RGB lights — cool vibe"},
}

# -----------------------------
# NEW FEATURE: Daily rewards
# -----------------------------
def DailyReward():
    global daily_reward_claimed
    if not daily_reward_claimed:
        print("\n🎁 DAILY REWARD UNLOCKED! 🎁")
        print("+20 points, +10 energy")
        state["points"] += 20
        state["energy"] = min(100, state["energy"] + 10)
        daily_reward_claimed = True
    else:
        print("\nYou've already claimed your daily reward today 😊")


def GreetUser():
    print("✨ Welcome to GenZ Life Simulator — Room Edition ✨")
    while True:
        name = input("Name: ").strip()
        if name:
            break
        print("Please enter a valid name.")
    while True:
        age = input("Age: ").strip()
        if age.isdigit():
            break
        print("Age must be a number.")
    user["name"] = name
    user["age"] = int(age)
    print(f"Nice to meet you, {name}! Let's vibe 💫")
    
    DailyReward()  # NEW: Give daily reward on login


def GetResponse(text):
    msg = text.lower().strip()
    for key in responses:
        if key in msg:
            return responses[key]
    return "Hmm... idk what to say 🤔"


def ShowStatus():
    print("\n--- STATUS ---")
    print(f"Energy: {state['energy']} / 100")
    print(f"Mood: {state['mood']}")
    print(f"Points: {state['points']}")
    print(f"Room Level: {state['room_level']} → {room_descriptions.get(state['room_level'], 'Top-tier room! ✨')}")
    print(f"Decor Items: {', '.join(i.title() for i in state['decor_items']) if state['decor_items'] else 'None'}")
    print("----------------\n")


def ShopMenu():
    print("\n🛒 ROOM DECOR SHOP 🛋️")
    for item, info in shop_items.items():
        owned = "(owned)" if item in state["decor_items"] else f"{info['cost']} pts"
        print(f"{item.title():12} → {owned:10} | {info['desc']}")

    while True:
        choice = input("Buy what (or 'exit')? ").strip().lower()
        if choice == "exit":
            return "Leaving shop..."
        if choice not in shop_items:
            print("Invalid item.")
            continue
        if choice in state["decor_items"]:
            print("You already have that.")
            continue

        cost = shop_items[choice]["cost"]
        if state["points"] < cost:
            print("Not enough points 😭")
            return "Shop closed."

        state["points"] -= cost
        state["decor_items"].append(choice)
        state["room_level"] += 1
        state["mood"] = "happy"

        # NEW FEATURE: Achievement messages
        if state["room_level"] == 3:
            print("🔥 Achievement Unlocked: Cozy Starter!")
        elif state["room_level"] == 5:
            print("✨ Achievement: Aesthetic Expert!")
        elif state["room_level"] == 7:
            print("🌟 Achievement: Room Influencer Status!")

        return f"Purchased {choice.title()}!"


# -----------------------------
# NEW FEATURE: Mood effects on points
# -----------------------------
def MoodMultiplier():
    mood = state["mood"]
    if mood == "happy":
        return 1.2
    if mood == "productive":
        return 1.1
    if mood == "sad":
        return 0.8
    return 1.0


# -----------------------------
# NEW FEATURE: Relax Action
# -----------------------------
def RelaxAction():
    state["energy"] = min(100, state["energy"] + 5)
    state["mood"] = "calm"
    return "You relaxed 🌿 +5 energy, mood → calm."


def UpdateStats(a):

    # ENERGY ALERT
    if state["energy"] <= 0:
        state["energy"] = 100
        state["mood"] = "rested"
        return "You passed out from exhaustion 😭 You auto-slept. Energy → 100."

    if a == "study":
        if state["energy"] < 15:
            return "Too tired to study 😵"
        state["energy"] -= 20
        gain = int(10 * MoodMultiplier())
        state["points"] += gain
        state["mood"] = "productive"
        return f"Studied 📚 -20 energy, +{gain} points."

    if a == "eat":
        gain = min(15, 100 - state["energy"])
        state["energy"] += gain
        state["mood"] = "satisfied"
        return f"Ate 🍔 +{gain} energy."

    if a == "play":
        if state["energy"] < 10:
            return "Not enough energy to play!"
        state["energy"] -= 10
        gain = int(5 * MoodMultiplier())
        state["points"] += gain
        state["mood"] = "excited"
        return f"Played 🎮 -10 energy, +{gain} points."

    if a == "relax":  # NEW
        return RelaxAction()

    if a == "sleep":
        state["energy"] = 100
        state["mood"] = "rested"
        return "Slept 😴 Energy restored."

    if a == "shop":
        return ShopMenu()

    return "Action not recognized."


def EndGame():
    print("\n=== GAME SUMMARY ===")
    ShowStatus()
    print("Thanks for playing 💖")


def activity_menu():
    while True:
        ShowStatus()
        cmd = input("(study/eat/play/relax/sleep/shop/quit): ").strip().lower()
        if cmd in ("quit", "exit"):
            break
        print(UpdateStats(cmd))
        time.sleep(0.4)


def conversation():
    print("Chat with your AI bestie 💬")
    while True:
        msg = input("> ").strip().lower()
        if not msg:
            continue

        if "play" in msg:
            if input("Play the game? (yes/no): ").startswith("y"):
                activity_menu()
                continue

        if msg in ("bye", "exit", "quit"):
            print("Aww okay bestie ❤️")
            EndGame()
            break

        print(GetResponse(msg))


if __name__ == "__main__":
    GreetUser()
    conversation()
